<a href="https://colab.research.google.com/github/Tanaifen/um-e-com/blob/main/streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
from datetime import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# --- Google Sheets Setup ---
SCOPE = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
CREDS_FILE = "boreal-card-463314-j5-33dd2c643d0c.json"  # your credentials file
SPREADSHEET_ID = "1WyByWq6jRSKFSXrvXnFCRriC7gDiPdzyFuGBfsG8q2Y"  # your actual Sheet ID

def save_to_google_sheet(data):
    creds = ServiceAccountCredentials.from_json_keyfile_name(CREDS_FILE, SCOPE)
    client = gspread.authorize(creds)
    sheet = client.open_by_key(SPREADSHEET_ID).sheet1
    sheet.append_row(data)

def load_data():
    creds = ServiceAccountCredentials.from_json_keyfile_name(CREDS_FILE, SCOPE)
    client = gspread.authorize(creds)
    sheet = client.open_by_key(SPREADSHEET_ID).sheet1
    records = sheet.get_all_records(expected_headers=[
        "timestamp", "age", "gender", "shop_freq", "satisfaction",
        "value_factors", "time_saving", "tech_issues", "avoid_online",
        "avoid_reasons", "barriers", "trust_payment", "encouragement",
        "recommendation", "trust_features", "comments"
    ])
    return pd.DataFrame(records)

# --- Streamlit Setup ---
st.set_page_config(page_title="E-Commerce Survey", layout="centered")
st.sidebar.title("🔍 Navigation")
mode = st.sidebar.selectbox("Select Mode", ["📋 Take Survey", "📊 Admin Dashboard"])

# --- Survey Mode ---
if mode == "📋 Take Survey":
    st.title("🛒 E-Commerce Efficiency & Satisfaction Survey 🛒")
    st.markdown("""
    ### 🎓 Survey Introduction
    This survey is part of the research project titled:
    **"Quantitative Evaluation of E-Commerce Efficiency and Customer Satisfaction Using Machine Learning Techniques."**
    """)

    st.markdown("""
    **Tan Ai Fen (22115538)**<br>
    **Postgraduate Student**<br>
    **Faculty of Computer Science and Information Technology**<br>
    **University of Malaya**
    """, unsafe_allow_html=True)

    st.markdown("---")

    # Section 1
    st.header("Section 1: Basic Information")
    age = st.selectbox("Age:", ["<18", "18–25", "26–35", "36–50", "50+"])
    gender = st.radio("Gender:", ["Male", "Female", "Prefer not to say", "Other"])
    shop_freq = st.radio("How often do you shop online?", ["Daily", "Weekly", "Monthly", "Rarely", "Never"])

    # Section 2
    st.header("Section 2: User Perception & Value")
    satisfaction = st.slider("Q1. Satisfaction with online shopping experience?", 1, 10)
    value_factors = st.multiselect("Q2. What do you value most?",
        ["Price", "Product quality", "Fast delivery", "Easy return policy",
         "Good customer service", "Secure payment", "Easy-to-use website/app"])
    time_saving = st.radio("Q3. Online shopping saves time:",
        ["Strongly Disagree", "Disagree", "Neutral", "Agree", "Strongly Agree"])
    tech_issues = st.radio("Q4. Frequency of technical issues:",
        ["Very Often", "Sometimes", "Rarely", "Never"])

    # Section 3
    st.header("Section 3: Barriers & Non-Adoption")
    avoid_online = st.radio("Q5. Avoided online shopping before?", ["Yes", "No"])
    avoid_reasons = []
    if avoid_online == "Yes":
        avoid_reasons = st.multiselect("If YES, why?",
            ["Trust issues", "Prefer in-store", "Complicated process",
             "Delivery takes too long", "Payment issues"])
    barriers = st.multiselect("Q6. Barriers to shopping online more often:",
        ["Hidden costs", "Lack of trust", "Long delivery time",
         "Prefer to see the item", "Poor customer service"])
    trust_payment = st.slider("Q7. Trust in online payment systems?", 1, 10)

    # Section 4
    st.header("Section 4: Suggestions & Recommendations")
    encouragement = st.multiselect("Q8. What would encourage more online shopping?",
        ["Faster shipping", "Better return process", "Lower prices",
         "Live chat/help", "Verified reviews", "Secure payment options"])
    recommendation = st.slider("Q9. Recommend online shopping to friends?", 0, 10)
    trust_features = st.multiselect("Q10. Features to increase trust:",
        ["Seller ratings", "Return centers", "Secure checkout", "Verified reviews"])
    additional_comments = st.text_area("Q11. Additional comments or suggestions?")

    if st.button("Submit Survey"):
        row = [
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            age, gender, shop_freq, satisfaction,
            ', '.join(value_factors), time_saving, tech_issues, avoid_online,
            ', '.join(avoid_reasons) if avoid_online == "Yes" else "",
            ', '.join(barriers), trust_payment,
            ', '.join(encouragement), recommendation,
            ', '.join(trust_features), additional_comments
        ]
        try:
            save_to_google_sheet(row)
            st.success("✅ Your response has been saved to Google Sheets.")
        except Exception as e:
            st.error(f"❌ Failed to save: {e}")

# --- Admin Dashboard Mode ---
elif mode == "📊 Admin Dashboard":
    st.title("📊 Admin Dashboard - Survey Results")

    try:
        df = load_data()

        if df.empty:
            st.warning("⚠️ No responses submitted yet.")
        else:
            st.subheader("📄 Raw Data")
            st.dataframe(df)

            st.subheader("🛍️ Shopping Frequency")
            st.bar_chart(df["shop_freq"].value_counts())

            st.subheader("📈 Satisfaction Scores")
            st.line_chart(df["satisfaction"])

            st.subheader("🔐 Trust in Payment Systems")
            st.bar_chart(df["trust_payment"])

            st.subheader("📣 Recommendation Likelihood")
            st.line_chart(df["recommendation"])

    except Exception as e:
        st.error(f"❌ Failed to load dashboard: {e}")